## 01a Taxi Data - Preprocessing

In this first notebook, the main dataframe is preprocessed. This includes:
- Loading the data
- Changing its datatypes
- Dropping trivial or null columns/rows
- Splitting the data into subsets and saving them for further work

As the original data is rather large in size, it is not included in the repository. Instead, when wanting to run this notebook, it is necessary to:
1. Download the data from https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew#column-menu where you specify via Actions->Query Data->Trip Start Timestamp // In Between // 2022 Jan 01 12:00:00 AM AND 2022 Dec 31 11:59:59 PM
2. Rename the file to "taxidata"
3. Run the code cell below and add the data to the following directory

In [1]:
import os
# this directory for the original data file
os.makedirs('./data', exist_ok=True)
# this directory to later save the prepared data
os.makedirs('./data/prep', exist_ok=True)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

In [3]:
# Data file not included in the project, needs to be downloaded individually. This step can take a few minutes due to size of the original file
df = pd.read_csv("data/taxidata.csv")

In [4]:
df.head(1)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,4404c6835b9e74e9f74d70f235200a8ce09db14a,7e179f8ef66ae99ec2d1ec89224e0b7ee5469fe5627f6d...,2022-12-31T23:45:00.000,2023-01-01T00:15:00.000,2081.0,4.42,NaN,NaN,2.0,3.0,...,0.0,20.5,Prcard,Flash Cab,42.001571,-87.695013,POINT (-87.6950125892 42.001571027),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)


After taking an initial look at the data and dicussing what columns may be relevant for later on, we decided on dropping a few columns which to us appeared to be trivial for the given context:

In [5]:
df_coldrop = df.drop(['pickup_community_area', 'dropoff_community_area',
                      'fare', 
                      # 'tips', 
                      'tolls', 
                      'extras',
                      # 'payment_type', 
                      # 'company'
                     ], axis=1)
df_coldrop.head(1)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,tips,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,4404c6835b9e74e9f74d70f235200a8ce09db14a,7e179f8ef66ae99ec2d1ec89224e0b7ee5469fe5627f6d...,2022-12-31T23:45:00.000,2023-01-01T00:15:00.000,2081.0,4.42,NaN,NaN,0.0,20.5,Prcard,Flash Cab,42.001571,-87.695013,POINT (-87.6950125892 42.001571027),41.965812,-87.655879,POINT (-87.6558787862 41.96581197)


In [6]:
# Convert time types to check if entries are from correct range
df_coldrop["trip_start_timestamp"] = pd.to_datetime(df_coldrop["trip_start_timestamp"])
df_coldrop["trip_end_timestamp"] = pd.to_datetime(df_coldrop["trip_end_timestamp"])

# In range of 2022:
print(f"Min date: {df_coldrop['trip_start_timestamp'].min()}")
print(f"Max date: {df_coldrop['trip_start_timestamp'].max()}")

Min date: 2022-01-01 00:00:00
Max date: 2022-12-31 23:45:00


Make sure these are the right dates, here it should say 2022-01-01 00:00:00 and 2022-12-31 23:45:00. 

In [7]:
# Look into null values
print(f"General shape of dataframe: {df_coldrop.shape}")
print(df_coldrop.isna().sum())

General shape of dataframe: (6382425, 18)
trip_id                             0
taxi_id                           354
trip_start_timestamp                0
trip_end_timestamp                212
trip_seconds                     1465
trip_miles                         56
pickup_census_tract           3758594
dropoff_census_tract          3707094
tips                             3536
trip_total                       3536
payment_type                        0
company                             0
pickup_centroid_latitude       511551
pickup_centroid_longitude      511551
pickup_centroid_location       511551
dropoff_centroid_latitude      597931
dropoff_centroid_longitude     597931
dropoff_centroid_location      597931
dtype: int64


We can see that there are some rows for "trip_start_timestamp" and "taxi_id" are null. As this only concerns very few rows (~560 of 6 mil), and we later on want to make computations where a valid end time is necessary, we decided to drop these.

In [8]:
df_coldrop.dropna(subset=['trip_end_timestamp', 'taxi_id'], axis=0, inplace=True)
df_coldrop.isna().sum()

trip_id                             0
taxi_id                             0
trip_start_timestamp                0
trip_end_timestamp                  0
trip_seconds                     1253
trip_miles                         56
pickup_census_tract           3758103
dropoff_census_tract          3706530
tips                             3533
trip_total                       3533
payment_type                        0
company                             0
pickup_centroid_latitude       511096
pickup_centroid_longitude      511096
pickup_centroid_location       511096
dropoff_centroid_latitude      597368
dropoff_centroid_longitude     597368
dropoff_centroid_location      597368
dtype: int64

Next, we want to check and drop entrie where "trip_seconds", "trip_total" or "trip_miles" are equal to 0.

In [9]:
print(f"Number of entries for which 'trip_seconds' is less or equal to 0: {len(df_coldrop[(df_coldrop['trip_seconds'] <= 0)])}")
print(f"Number of entries for which 'trip_miles' is less or equal to 0: {len(df_coldrop[(df_coldrop['trip_miles'] <= 0)])}")
print(f"Number of entries for which 'trip_total' is less or equal to 0: {len(df_coldrop[(df_coldrop['trip_total'] <= 0)])}")

# Total (no duplicates)
print(f"-----\nTotal: {len(df_coldrop[(df_coldrop['trip_seconds'] <= 0) | (df_coldrop['trip_miles'] <= 0) | (df_coldrop['trip_total'] <= 0)])}")

df_coldrop[(df_coldrop['trip_seconds'] <= 0) | (df_coldrop['trip_miles'] <= 0) | (df_coldrop['trip_total'] <= 0)][5000:10000:500]

Number of entries for which 'trip_seconds' is less or equal to 0: 155315
Number of entries for which 'trip_miles' is less or equal to 0: 795007
Number of entries for which 'trip_total' is less or equal to 0: 5667
-----
Total: 803813


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,tips,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
42177,076fca0f85d8de7cebd8302e2f794c1399150aa2,6607f4be1b8737ffe33ac1f71d10d1132541d86cd00c1d...,2022-12-29 19:00:00,2022-12-29 19:15:00,770.0,0.0,NaN,NaN,2.37,15.01,Mobile,Flash Cab,41.965812,-87.655879,POINT (-87.6558787862 41.96581197),41.922686,-87.649489,POINT (-87.6494887289 41.9226862843)
46872,0656608294696d4c1330c2b04fe03bfbe8c31c0c,16a529a73f019759aa8f1b563ab8511b26a91d65659188...,2022-12-29 14:15:00,2022-12-29 14:15:00,540.0,0.0,1.703108e+10,1.703108e+10,2.00,8.75,Credit Card,Taxi Affiliation Services,41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),41.891972,-87.612945,POINT (-87.6129454143 41.8919715078)
51808,00af44e86d74411c033bb045dcd54b6bb396d62f,b5bf5d282fa4191c68fe6552ccd45134162eacba69b016...,2022-12-29 08:15:00,2022-12-29 08:45:00,1560.0,0.0,NaN,NaN,10.30,51.05,Credit Card,Taxi Affiliation Services,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.980264,-87.913625,POINT (-87.913624596 41.9802643146)
56531,a51bb770554e5833df5124cf5f3812ca86fb2309,c3bb697ebd10de692888e34f4f3c1ad35d7d5bc0bccc15...,2022-12-28 18:15:00,2022-12-28 18:15:00,480.0,0.0,1.703108e+10,1.703108e+10,0.00,6.25,Cash,"Taxicab Insurance Agency, LLC",41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.907520,-87.626659,POINT (-87.6266589003 41.9075200747)
61530,3c34314f321854e5d14c390a3c42e3fd598c5b6b,ccc7e08aa6d30e16647614e562aea896f7375110375898...,2022-12-28 13:30:00,2022-12-28 13:45:00,628.0,0.0,NaN,NaN,3.19,17.06,Mobile,Flash Cab,NaN,NaN,NaN,NaN,NaN,NaN
66184,c8d97be5389df036fd756a8b6fa075875cabb562,5f395c33ef34f1545aa60c48463f313795e1d872ea9acc...,2022-12-28 07:00:00,2022-12-28 07:15:00,816.0,0.0,NaN,NaN,10.00,26.61,Mobile,Flash Cab,41.947792,-87.683835,POINT (-87.6838349425 41.9477915865),41.899602,-87.633308,POINT (-87.6333080367 41.899602111)
70728,6d1ee9a9c319e7f8a7878bbeb6f08d3a9dc2bab8,4364d99b1f14eb6e63596a20a1a936c805239d09b4675e...,2022-12-27 18:00:00,2022-12-27 18:00:00,420.0,0.0,NaN,NaN,2.00,9.00,Credit Card,Taxi Affiliation Services,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.944227,-87.655998,POINT (-87.6559981815 41.9442266014)
75321,0463852e513d5ad54151376392aaca455012449c,83a83c946de3f1d72d9764569c64d39eb97fc7219966be...,2022-12-27 13:15:00,2022-12-27 13:30:00,1020.0,0.0,1.703128e+10,1.703108e+10,0.00,12.00,Cash,"Taxicab Insurance Agency, LLC",41.879255,-87.642649,POINT (-87.642648998 41.8792550844),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809)
80222,f48fde12fcc0cf01f88f17ee0aaaed79b32ca8cf,bf0b987f41a8c368a072da293f9fb1aecc8a39763ab6a4...,2022-12-27 01:30:00,2022-12-27 01:30:00,4.0,0.0,NaN,NaN,0.00,73.00,Cash,Flash Cab,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.980264,-87.913625,POINT (-87.913624596 41.9802643146)
84524,9b7db8d486466885cbb3aab6c75960d904be4aaf,f8f44c9d76773e3c761356078d786b151d2f206685454f...,2022-12-26 17:00:00,2022-12-26 17:00:00,0.0,0.0,1.703133e+10,1.703133e+10,0.00,3.25,Cash,Taxi Affiliation Services,41.859350,-87.617358,POINT (-87.6173580061 41.859349715),41.859350,-87.617358,POINT (-87.6173580061 41.859349715)


Trips that do have any one of these three features recorded with values larger than 0, we keep. Entries that do not have values larger than 0 for any of the three features will be dropped, as a trip with 0 seconds, no miles or 0 revenue gained, we consider outliers. 

In [10]:
# Only save entries whose values are larger than 0
df_coldrop = df_coldrop[(df_coldrop['trip_seconds'] > 0) & (df_coldrop['trip_miles'] > 0) & (df_coldrop['trip_total'] > 0)]

Now sorting after the trip start times and using the "taxi_id" as the index column, we searched for other faulty data. We introduced new variable "time_inbetween_trips" to check for other faulty data.

In [11]:
# Sort after taxi id and start time
df_taxitime = df_coldrop.sort_values(by=['taxi_id', 'trip_start_timestamp'])

df_taxitime = df_taxitime.reset_index(drop=True)

df_taxitime.head(3)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,tips,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,30809ffcd766aea54241de95ec4cd320aa76172c,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,2022-01-06 00:00:00,2022-01-06 00:30:00,2280.0,1.9,1.703198e+10,NaN,20.0,136.50,Credit Card,Taxi Affiliation Services,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),NaN,NaN,NaN
1,5e97776a870de5fb393254de41a874a6c769236b,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,2022-01-06 21:15:00,2022-01-06 21:30:00,1500.0,1.2,NaN,NaN,3.5,56.50,Credit Card,Taxi Affiliation Services,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.857184,-87.620335,POINT (-87.6203346241 41.8571838585)
2,44563c0da23b63739a99daf82be74aa2665d1958,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,2022-01-06 23:45:00,2022-01-07 00:15:00,1260.0,0.7,NaN,NaN,0.0,35.25,Cash,Taxi Affiliation Services,41.792592,-87.769615,POINT (-87.7696154528 41.7925923603),NaN,NaN,NaN


In [12]:
# Calculate time between end of first row with the start time of next row
# Explicit type conversion (should not be needed since done above)
df_taxitime['trip_start_timestamp'] = pd.to_datetime(df_taxitime['trip_start_timestamp'])
df_taxitime['trip_end_timestamp'] = pd.to_datetime(df_taxitime['trip_end_timestamp'])

# Calculate the time between end time and start time of the next row
df_taxitime["time_inbetween_trips"] = df_taxitime.groupby('taxi_id')['trip_start_timestamp'].shift(-1) - df_taxitime['trip_end_timestamp']

In [13]:
df_taxitime.head(4)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,tips,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,time_inbetween_trips
0,30809ffcd766aea54241de95ec4cd320aa76172c,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,2022-01-06 00:00:00,2022-01-06 00:30:00,2280.0,1.9,1.703198e+10,NaN,20.0,136.50,Credit Card,Taxi Affiliation Services,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),NaN,NaN,NaN,0 days 20:45:00
1,5e97776a870de5fb393254de41a874a6c769236b,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,2022-01-06 21:15:00,2022-01-06 21:30:00,1500.0,1.2,NaN,NaN,3.5,56.50,Credit Card,Taxi Affiliation Services,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.857184,-87.620335,POINT (-87.6203346241 41.8571838585),0 days 02:15:00
2,44563c0da23b63739a99daf82be74aa2665d1958,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,2022-01-06 23:45:00,2022-01-07 00:15:00,1260.0,0.7,NaN,NaN,0.0,35.25,Cash,Taxi Affiliation Services,41.792592,-87.769615,POINT (-87.7696154528 41.7925923603),NaN,NaN,NaN,0 days 19:45:00
3,26bc737e6c416473c6b44778c9af19e91f1b2f42,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,2022-01-07 20:00:00,2022-01-07 20:15:00,1380.0,0.9,NaN,NaN,10.8,53.55,Credit Card,Taxi Affiliation Services,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),0 days 02:15:00


Trips happen at a 15 to 30min interval. Trips that are consecutive should have some spare time in between them, hence we drop entries where this is not the case and we have negative/<= 0 time in between.

In [14]:
# Check for false entries regarding "time_inbetween_trips", e.g. such that are negative
print(f"Entries with negative time between trips: {len(df_taxitime[df_taxitime['time_inbetween_trips'].dt.total_seconds() < 0])}")

# Only keep positive times
df_taxitime2 = df_taxitime[df_taxitime["time_inbetween_trips"].dt.total_seconds() >= 0]
df_taxitime2.head(2)

Entries with negative time between trips: 89531


,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,tips,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location,time_inbetween_trips
0,30809ffcd766aea54241de95ec4cd320aa76172c,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,2022-01-06 00:00:00,2022-01-06 00:30:00,2280.0,1.9,1.703198e+10,NaN,20.0,136.5,Credit Card,Taxi Affiliation Services,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),NaN,NaN,NaN,0 days 20:45:00
1,5e97776a870de5fb393254de41a874a6c769236b,0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...,2022-01-06 21:15:00,2022-01-06 21:30:00,1500.0,1.2,NaN,NaN,3.5,56.5,Credit Card,Taxi Affiliation Services,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),41.857184,-87.620335,POINT (-87.6203346241 41.8571838585),0 days 02:15:00


In [15]:
# Null values that are left:
df_taxitime2.isna().sum()
### What else should be dropped?

trip_id                             0
taxi_id                             0
trip_start_timestamp                0
trip_end_timestamp                  0
trip_seconds                        0
trip_miles                          0
pickup_census_tract           3258352
dropoff_census_tract          3189731
tips                                0
trip_total                          0
payment_type                        0
company                             0
pickup_centroid_latitude       451153
pickup_centroid_longitude      451153
pickup_centroid_location       451153
dropoff_centroid_latitude      504438
dropoff_centroid_longitude     504438
dropoff_centroid_location      504438
time_inbetween_trips                0
dtype: int64

Lastly, we want to encode string columns?